In [5]:
import numpy as np
from PIL import Image, ImageFilter

In [6]:
# Open image and make into Numpy array
im = Image.open('./Square detection results/ppl1.png').convert('RGB')
na = np.array(im)
orig = na.copy()    # Save original

In [7]:
# Median filter to remove outliers
im = im.filter(ImageFilter.MedianFilter(3))

In [8]:
# Find X,Y coordinates of all yellow pixels
yellowY, yellowX = np.where(np.all(na==[0, 0, 255],axis=2))
print(yellowX)
#yellowY2, yellowX2= np.where(np.all(na==[0, 0, 255],axis=-2))
zipped = np.column_stack((yellowX,yellowY))
print("xipp",zipped)

[718 719 720 ... 433 434 435]
xipp [[718  22]
 [719  22]
 [720  22]
 ...
 [433  92]
 [434  92]
 [435  92]]


In [9]:
uniqueY = list(yellowY)
print(uniqueY)
uniqueX= list(set(yellowX))
print(uniqueX)
uniquesetX={uniqueX[0]}

uniquesetY={uniqueY[0]}

[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28,

In [10]:
for x in range(0,len(uniqueX)-1):
  if(uniqueX[x+1]-uniqueX[x]>=2):
    uniquesetX.add(uniqueX[x])
    uniquesetX.add(uniqueX[x+1])
uniquesetX.add(uniqueX[len(uniqueX)-1])
uniquesetX=sorted(uniquesetX)
print(uniquesetX)
uniquesetY=list()
for a in uniquesetX:
  for x,y in zipped:
    if(a==x):
      uniquesetY.append(y)
  print("nextx",a)
  print(uniquesetY)

[69, 130, 245, 298, 378, 436, 555, 612, 717, 773, 862, 919]
nextx 69
[29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88]
nextx 130
[29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88]
nextx 245
[29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 

In [11]:
print(uniquesetY)

[29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,

In [12]:
uniqueset2Y=list()
uniqueset2Y.append(uniquesetY[0])

In [13]:
for x in range(0,len(uniquesetY)-1):
  if(uniquesetY[x+1]-uniquesetY[x]!=1):
    uniqueset2Y.append(uniquesetY[x])
    uniqueset2Y.append(uniquesetY[x+1])
uniqueset2Y.append(uniquesetY[len(uniquesetY)-1])
print("uniqesetY",uniqueset2Y)
uniqueset3Y=list()
for x in range(0,len(uniqueset2Y),4):
  uniqueset3Y.append(uniqueset2Y[x])
  uniqueset3Y.append(uniqueset2Y[x+1])
print("tellmefinal")
print(uniqueset3Y)
#tried rotating
#print("roatted")

#rotimg = im.rotate(90)
#na2=np.array(rotimg)
#yellowrY, yellowrX = np.where(np.all(na2==[0, 0, 255],axis=2))
#uniqueX= list(set(yellowrX))
#print(uniqueX)
# for y in range(0,len(uniqueY)):
#   for x in range(0,len(uniquesetX)):
    #if(na[x][y]==[0,0,255]):
    ##  uniquesetY.add(y)
  



#uniquesetY=set()
#for y in range(0,len(uniqueY)-1):
#  if(uniqueY[y+1]-uniqueY[y]>=2):
  #  print('hyi')
  #  uniquesetY.add(uniqueY[y])
#print(uniquesetY)
#if()
# top, bottom = yellowY[0], yellowY[-1]
# left, right = yellowX[0], yellowX[-1]
# print(top,bottom,left,right)

# # Extract Region of Interest from unblurred original
#ROI = orig[top:bottom, left:right]

# Image.fromarray(ROI).save('result.png')

uniqesetY [29, 88, 29, 88, 26, 77, 26, 77, 35, 91, 35, 91, 30, 85, 30, 85, 23, 77, 23, 77, 31, 86, 31, 86]
tellmefinal
[29, 88, 26, 77, 35, 91, 30, 85, 23, 77, 31, 86]


In [ ]:
import cv2
img = cv2.imread('./img/ppl1.png')
for i in range(0,len(uniquesetX)-1,2):
  a=uniqueset3Y[i]
  b=uniqueset3Y[i+1]
  c=uniquesetX[i]
  d=uniquesetX[i+1]
  print(i,a,b,c,d)
  face=img[a:b,c:d]

cv2.imwrite("./Square detection results/test.jpg", face)
  
  
  
# face = img[30:87, 70:129]
cv2.imwrite("./ans1.jpg", face)
cv2.waitKey(0)

0 29 88 69 130
2 26 77 245 298
4 35 91 378 436
6 30 85 555 612
8 23 77 717 773
10 31 86 862 919
